In [ ]:
from google import genai
from google.genai import types
import json

In [ ]:
with open("secrets.json") as f:
    secrets = json.load(f)

api_key = secrets["api_key"]

In [3]:
def generate(input_text, chapter, section, file_name):
    client = genai.Client(
        api_key=api_key,
    )

    model = "gemini-2.0-flash-thinking-exp-01-21"

    prompt = f"""Bạn có nhiệm vụ tạo ra 10 câu hỏi cho một đoạn văn bản được người dùng đưa ra để tạo ra bô dataset huấn luyện một hệ thống RAG. Các câu hỏi bạn đưa ra CẦN PHẢI có độ khó đa dạng từ thấp đến cao, không bị trùng lặp và không được để lộ là bạn đang hỏi về đoạn văn bản đó. Một câu hỏi được đưa ra phải có dạng 1 đoạn văn, các câu hỏi được ngăn cách bởi xuống dòng. Câu trả lời cưa bạn CHỈ được chứa các câu hỏi và CHỈ được trả lời bằng Tiếng Việt, KHÔNG ĐƯỢC PHÉP nói gì thêm trong câu trả lời.
    
    Đoạn văn bản:
    {input_text}
    
    Thông tin thêm:
    - Nguồn gốc văn bản:
        + Tên: {file_name}
        + Chương: {chapter}
        + Điều: {section}
    - Đây là văn bản quy chế của Đại học Bách khoa Hà Nội, bạn cần tạo ra các câu hỏi liên quan đến quy chế học tập của sinh viên.
    """

    response = client.models.generate_content(
        model=model,
        contents=[prompt],
    )
    print(f"Response: {response.text}")

    return list(response.text.split("\n"))


In [2]:
import pandas as pd

In [5]:
file = "data/final.csv"

df = pd.read_csv(file)

In [29]:
for i in range(148, 149):
    print(f"Processing row {i+1}/{len(df)}")
    df.at[i, "questions"] = generate(df.at[i, "content"], df.at[i, "chapter"], df.at[i, "section"], df.at[i, "from"])

Processing row 149/149
Response: Năm học nào quy định mới về đào tạo chính thức được áp dụng cho sinh viên?

Những sinh viên nhập học vào năm 2022 cần đặc biệt lưu ý đến điểm nào trong quy chế mới này liên quan đến việc xét tốt nghiệp?

Có sự khác biệt nào trong quy định về việc học cùng lúc hai chương trình giữa các khóa tuyển sinh khác nhau không? Nếu có, sự khác biệt đó là gì?

Những sinh viên thuộc khóa nào và tốt nghiệp chương trình kỹ sư với thời gian đào tạo 5 năm sẽ được cấp bằng theo quyết định cụ thể nào của trường?

Điều gì sẽ xảy ra với các quy định về đào tạo đã được ban hành trước đây nhưng lại không thống nhất với quy chế mới nhất?

Trong quy chế mới, những nội dung nào không được áp dụng ngay từ năm học 2023-2024 mà lại có thời gian áp dụng khác biệt?

Sinh viên có thể tìm hiểu thông tin chi tiết về quyết định số 66/QĐ-ĐHBK-ĐT ở đâu và quyết định này liên quan đến vấn đề gì trong đào tạo?

Quy chế mới này có những điều chỉnh nào đáng chú ý về tiêu chuẩn xét tốt nghiệp s

In [18]:
df['questions'] = df['questions'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df = df.explode("questions", ignore_index=True)
df['questions'] = df['questions'].apply(lambda x: None if x == "" else x)
df = df.dropna(subset=["questions"])

In [19]:
df

,id,from,chapter,section,content,questions
0,937148,Yêu cầu về Ngoại Ngữ 2021,QUY ĐỊNH: Phân loại trình độ đầu vào và chương...,Điều 1. Phạm vi và đối tượng áp dụng,1. Văn bản này quy định về công tác tổ chức đá...,Đâu là những yếu tố chính thường được đề cập đ...
2,937148,Yêu cầu về Ngoại Ngữ 2021,QUY ĐỊNH: Phân loại trình độ đầu vào và chương...,Điều 1. Phạm vi và đối tượng áp dụng,1. Văn bản này quy định về công tác tổ chức đá...,Những quy định về ngoại ngữ trong môi trường đ...
4,937148,Yêu cầu về Ngoại Ngữ 2021,QUY ĐỊNH: Phân loại trình độ đầu vào và chương...,Điều 1. Phạm vi và đối tượng áp dụng,1. Văn bản này quy định về công tác tổ chức đá...,"Ngoài việc quy định về chuẩn đầu ra, một văn b..."
6,937148,Yêu cầu về Ngoại Ngữ 2021,QUY ĐỊNH: Phân loại trình độ đầu vào và chương...,Điều 1. Phạm vi và đối tượng áp dụng,1. Văn bản này quy định về công tác tổ chức đá...,"Trong hệ thống giáo dục đại học, việc phân loạ..."
8,937148,Yêu cầu về Ngoại Ngữ 2021,QUY ĐỊNH: Phân loại trình độ đầu vào và chương...,Điều 1. Phạm vi và đối tượng áp dụng,1. Văn bản này quy định về công tác tổ chức đá...,Những sinh viên thuộc hệ đào tạo chính quy tại...
...,...,...,...,...,...,...
2788,503781,Quy chế từ 2023,CHƯƠNG VI TỔ CHỨC THỰC HIỆN,Điều 47. Hiệu lực thi hành,1. Quy chế này được áp dụng từ năm học 2023-20...,"Trong quy chế mới, những nội dung nào không đư..."
2790,503781,Quy chế từ 2023,CHƯƠNG VI TỔ CHỨC THỰC HIỆN,Điều 47. Hiệu lực thi hành,1. Quy chế này được áp dụng từ năm học 2023-20...,Sinh viên có thể tìm hiểu thông tin chi tiết v...
2792,503781,Quy chế từ 2023,CHƯƠNG VI TỔ CHỨC THỰC HIỆN,Điều 47. Hiệu lực thi hành,1. Quy chế này được áp dụng từ năm học 2023-20...,Quy chế mới này có những điều chỉnh nào đáng c...
2794,503781,Quy chế từ 2023,CHƯƠNG VI TỔ CHỨC THỰC HIỆN,Điều 47. Hiệu lực thi hành,1. Quy chế này được áp dụng từ năm học 2023-20...,Việc quy chế mới có hiệu lực thi hành từ năm h...


In [20]:
df.to_csv("data/final_question.csv", index=False)